In [ ]:
import glob
import polars as pl
import pandas as pd
from ydata_profiling import ProfileReport, compare

In [3]:
from pages.code.lib import *
import streamlit as st
import plotly.express as px

# st.set_page_config(layout="wide")

image_path = 'logo.png'
image_base64 = get_image_base64(image_path)
html_content = get_image_html(image_base64)

# st.markdown(html_content, unsafe_allow_html=True)

filter_columns = ["Domain", "Area", "Element", "Item", "Unit"]

data = get_data()
data = clean_data(data)
data = join_data(data)
data["Filter"] = data[filter_columns].apply(lambda row: ' | '.join(row.values), axis=1)

selected_columns = data.drop(columns=["Year", "Value", "Unit", "Filter", "Area", "Flag Description"]).columns

#@st.cache_data(show_spinner = "Generating Forecast")
def get_forecast(data, steps = 5):
    forecast_df = data[["Year", "Value", "Filter"]].pivot_table(index = "Year", columns="Filter", values="Value", aggfunc="mean")
    forecast_df = forecast_df.reindex(range(forecast_df.index.min(), forecast_df.index.max() + steps + 1))
    forecast_df  = forecast_df.interpolate(mmethod='akima', axis="index", limit_direction = "both")
    forecast_df = forecast_df.reset_index().melt(id_vars="Year")
    forecast_df = forecast_df.merge(pd.DataFrame([dict(zip(filter_columns, values)) for values in forecast_df["Filter"].str.split("|").to_list()]), left_index=True, right_index=True).rename(columns={"value":"Value"})
    return forecast_df

data = get_forecast(data)
df = data[(data['Item'].str.contains('cocoa|chocolate', case=False, na=False)) & (data['Filter'].str.contains('export value', case=False, na=False))]

df.loc[(df['Item'].str.contains('cocoa beans', case=False, na=False)), "Value"] = df.loc[(df['Item'].str.contains('cocoa beans', case=False, na=False)), "Value"] * (1 - 0.3)
df.loc[(~df['Item'].str.contains('cocoa beans', case=False, na=False)), "Value"] = df.loc[(~df['Item'].str.contains('cocoa beans', case=False, na=False)), "Value"] * (1 - 0.02)
df.loc[(~df['Item'].str.contains('cocoa beans', case=False, na=False)), "Item"] = "Cocao Product"

tmp = df.drop(columns="Filter").groupby(by = ["Domain", "Area", "Element", "Item", "Year", "Unit"]).sum().reset_index()
#px.line(vis_data, x = "Year", y ="Value", color = "Area", title = val, height=700)
px.bar(tmp, x = "Year", y = "Value", color = "Item")

In [4]:
tmp = df.drop(columns="Filter").groupby(by = ["Domain", "Area", "Element", "Item", "Year", "Unit"]).sum().reset_index()
#px.line(vis_data, x = "Year", y ="Value", color = "Area", title = val, height=700)
px.bar(tmp, x = "Year", y = "Value", color = "Item")


In [ ]:
from PIL import Image

# Open an image file
image_path = 'logo.png'  # Change this to the path of your image file
image = Image.open(image_path)

# Save the image as PNG
output_path = 'logo.png'  # Change this to your desired output path
image.save(output_path, 'PNG')

Load each dataframe

In [ ]:
df_p = pd.read_csv("data/production.csv")
df_p

In [ ]:
df_l = pd.read_csv("data/land.csv")
df_l

In [ ]:
df_t = pd.read_csv("data/trade.csv",)
df_t

1. Same number of columns for each dataframe
2. Column names are the same
3. Number of rows are different


In [ ]:
kwargs = dict(samples=None, correlations=None, missing_diagrams=None, duplicates=None, interactions=None)

profile_t = ProfileReport(df_t, title = "Trade Data", **kwargs)
profile_p = ProfileReport(df_p, title = "Production Data", **kwargs)
profile_l = ProfileReport(df_l, title = "Land Data", **kwargs)


comparison_report = compare([profile_l, profile_p, profile_t])


comparison_report   #.to_file("comparison.html")

Based on the dashboard above we can start to get a good basic grasp on the datasets

1. Note - 90% of values are missing
2. Unit, Value, Flag, and Flag Description all have a high amount of missing values
3. Unit, Value, Flag, and Flag Description all have equal proportions of missing values

Remove missing values within the rows of the columns Unit, Value, Flag, and Flag Description for two reasons:

1. There are a high amount and there appears to be not way to determine these values at first glance
2. Q1 states that the realiablity of the values must be known

In [ ]:
df_l.drop(columns = ["Domain Code", "Area Code (M49)", "Element Code", "Item Code", "Flag", "Year Code", "Note"], inplace = True)
df_t.drop(columns = ["Domain Code", "Area Code (M49)", "Element Code", "Item Code (CPC)", "Flag", "Year Code", "Note"], inplace = True)
df_p.drop(columns = ["Domain Code", "Area Code (M49)", "Element Code", "Item Code (CPC)", "Flag", "Year Code", "Note"], inplace = True)

df_l.dropna(subset=["Unit", "Value", "Flag Description"], axis = 0, inplace = True)
df_t.dropna(subset=["Unit", "Value", "Flag Description"], axis = 0, inplace = True)
df_p.dropna(subset=["Unit", "Value", "Flag Description"], axis = 0, inplace = True)

In [ ]:
df_p["Area"].nunique()

In [ ]:
df_l["Area"].nunique()

In [ ]:
df_t["Area"].nunique()

Check dashboard again

In [ ]:
kwargs = dict(samples=None, correlations=None, missing_diagrams=None, duplicates=None, interactions=None)

profile_t = ProfileReport(df_t, title = "Trade Data", **kwargs)
profile_p = ProfileReport(df_p, title = "Production Data", **kwargs)
profile_l = ProfileReport(df_l, title = "Land Data", **kwargs)


comparison_report = compare([profile_l, profile_p, profile_t])


comparison_report

In [ ]:
df = pd.concat([df_t, df_p, df_l])

In [ ]:
# df_l["Element"] + df_l["Item"] + df_l["Unit"]
tmp = df
# tmp["Filter"] = (tmp["Item"] + " (" + tmp["Unit"] + ") - " +  tmp["Flag Description"])
# tmp = tmp.drop(columns=["Item", "Unit", "Flag Description"])
# tmp = tmp.drop(columns="Element").drop_duplicates()
# tmp

In [ ]:
tmp["filter"] = tmp[["Item", "Unit", "Flag Description"]].apply(lambda row: ' | '.join(row.values), axis=1)
tmp

In [ ]:
filtered_df = tmp[tmp["Item Unit"] == "Cocoa beans (t) - Official figure"]
px.line(filtered_df, x ="Year", y="Value", color="Area")

In [ ]:
import plotly.express as px

tmp = df_l.pivot_table(index=['Area', 'Year', "Flag Description", 'Item'] , columns="Unit", values='Value').reset_index()

px.line(tmp, x = "Year", y = "%", color="Area", facet_col="Flag Description")

In [ ]:
import plotly.express as px

tmp = df_l.pivot_table(index=['Area', 'Year', "Flag Description", 'Item'] , columns="Unit", values='Value').reset_index()

px.line(tmp, x = "Year", y = "%", color="Area", facet_col="Flag Description")

In [ ]:
import plotly.express as px

tmp = df_l.drop(columns=["Domain", "Domain Code", "Area Code (M49)", "Element Code", "Item Code", "Flag", "Year Code"])

exclude_columns = ["Area", "Year", "Value"]

key_column_values = [{key: val} for key in tmp.columns.to_list() for val in tmp[key].unique().tolist() if key not in exclude_columns]

# for key, val in key_column_values.items():

#     for val2 in val:

#         fig = px.line(tmp[tmp[key] == val])

key_column_values

In [ ]:
pl_df = pl.DataFrame(df_l).drop(["Domain", "Domain Code", "Area Code (M49)", "Element Code", "Item Code", "Flag", "Year Code", "Value"]).unique()

pl_df.

In [ ]:
tmp = df_l.drop(columns=["Domain", "Domain Code", "Area Code (M49)", "Element Code", "Item Code", "Flag", "Year Code"])

for col in tmp.columns.to_list():
    print(tmp[col].unique().tolist())

In [ ]:
tmp = df_l.drop(columns=["Domain", "Domain Code", "Area Code (M49)", "Element Code", "Item Code", "Flag", "Year Code"])

for group in tmp.groupby("Area"):
    print(group)
    break

In [ ]:
tmp = df_l.drop(columns=["Domain", "Domain Code", "Area Code (M49)", "Element Code", "Item Code", "Flag", "Year Code"])


comparison_report = compare([ProfileReport(group[1], tsmode=True, sortby="Year", title = f"{group[0]}") for group in tmp.groupby("Area")])
comparison_report 

In [ ]:
profile_t = ProfileReport(df_l, title = "Trade Data", tsmode = True, sortby = "Year")
profile_t

Awesome by just doing that we got rid of all the missing values, now we can start doing some nice visuals

In [ ]:
import plotly.express as px

df_l["Area"].unique().tolist()

In [ ]:
df_l[df_l["Unit"] == "1000 ha"]

In [ ]:
px.line(df_l, x = "Year", y = "Value", color = "Area", facet_row = "Unit")

In [ ]:
df_l

In [ ]:
tmp_df = df_l[df_l["Unit"] == "1000 ha"]

px.line(tmp_df, x = "Year", y = "Value", color = "Area",)

In [ ]:
df_l